In [ ]:
%matplotlib inline

import sys
sys.path.append('../')

import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('seaborn-white')
import pandas as pd
import seaborn as sns

import boolipy
api = boolipy.api.Api()

# Global variables

In [ ]:
FRIDHEMSPLAN = [59.334438, 18.029522]
CENTER = ",".join([str(x) for x in FRIDHEMSPLAN])
DIM = "400,500"
LIMIT = 1000
ENDPOINT = "sold"

parameters = {"center": CENTER, "dim": DIM, "limit": LIMIT}

In [3]:
data = api.get(endpoint=ENDPOINT, parameters=parameters, follow=True)
df = pd.DataFrame(data[0][ENDPOINT])

TypeError: Unicode-objects must be encoded before hashing

In [ ]:
from haversine import haversine, Unit

# define how do we want to change the data
def time_published(x): return x["soldDate"]-x["published"]
def source_name(x): return x["source"]["name"]
def price_increased(x): return x["soldPrice"] - x["listPrice"]
# convert location to tuple
def location_tuple(x): 
    loc = x['location']['position']
    return (loc['latitude'], loc['longitude'])
def dist_center(x, center = CENTER): 
    cloc = center
    if isinstance(center, str):
        cloc = tuple([float(l) for l in center.split(',')])

    # using default location
    #loc = x['location']['position']
    #c = (loc['latitude'], loc['longitude'])
    
    # using new loc_tuple
    c = x['loc_tuple']

    return haversine(c, cloc, unit=Unit.METERS)

# test distance
#d = {"location": {"position": {"latitude": 59.334438, "longitude": 18.029522}}}
#center = CENTER
#assert dist_center(d, center) == 0

In [ ]:
# apply all transformations
###########################
# transform data
df['loc_tuple'] = df.apply(location_tuple, axis=1)
df['soldDate'] = pd.to_datetime(df['soldDate'], format="%Y-%m-%d") 
df['published'] = pd.to_datetime(df['published']) 


# add new data
df['time_published'] = df.apply(time_published, axis=1)
df['source_name'] = df.apply(source_name, axis=1)
df['price_increased'] = df.apply(price_increased, axis=1)

# Plot apartments

In [ ]:
import folium

def get_popup_str(x): return 'size: {} </br> price: {} </br> date: {}'.format(x['livingArea'], x['soldPrice'], x['soldDate'])

m = folium.Map(
    location=FRIDHEMSPLAN,
    zoom_start=15
)

# center marker
folium.Marker(FRIDHEMSPLAN, popup='Center of the univers').add_to(m)

# apartments found
for index, row in df.iterrows():
    folium.Marker(row['loc_tuple'], popup=get_popup_str(row)).add_to(m)

m